In [3]:
from datetime import timedelta

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from google.cloud import bigquery

from etl.utils.utils_methods import load_params

In [2]:
params = load_params()
PROJECT_ID = params["bigquery"]["project_id"]
DATASET_ID = params["bigquery"]["dataset_id"]
print("Working project ID: ", PROJECT_ID)
print("Dataset ID: ", DATASET_ID)

Working project ID:  olist-data-warehouse-48014
Dataset ID:  olist_dw


In [ ]:
def run_query(query: str, project_id: str) -> pd.DataFrame:
    """
    Run a query to a specified BigQuery project.

    Args:
        query (str): SQL query to run.
        project_id: ID of the BigQuery project to query.

    Returns:
        (pd.DataFrame): pandas DataFrame with the results of the query.
    """
    client = bigquery.Client(project_id=project_id)
    query_job = client.query(query)
    results = query_job.result()
    return results.to_dataframe()

## Data Overview

In [ ]:
# `

## Order trends

### Order count per month

In [ ]:
qry = f"""
SELECT DATE_TRUNC(purchase_ts, MONTH) AS month,
COUNT(*) AS total_orders
FROM `{PROJECT_ID}.{DATASET_ID}.orders`
GROUP BY month
ORDER BY month
"""

df = run_query(query=qry, project_id=PROJECT_ID)

plt.figure(figsize=(12, 8))
sns.lineplot(data=df, x="month", y="total_orders")
plt.title("Total Orders per Month")
plt.show()

### Top 10 products categories

In [ ]:
qry = f"""
SELECT name, COUNT(*) AS items_sold
FROM `{PROJECT_ID}.{DATASET_ID}.order_items` o
JOIN `{PROJECT_ID}.{DATASET_ID}.products` p ON o.product_id = p.product_id
GROUP BY name
ORDER BY items_sold DESC
SELECT 10
"""

df = run_query(query=qry, project_id=PROJECT_ID)

plt.figure(figsize=(10, 6))
sns.barplot(data=df, x="items_sold", y="name", palette="viridis")
plt.title("Top 10 Categories by Items Sold")
plt.xlabel("Items Sold")
plt.ylabel("Category")
plt.show()

### Total revenue per category

In [ ]:
qry = f"""
SELECT name, SUM(o.price + o.freight_value) AS total_revenue
FROM `{PROJECT_ID}.{DATASET_ID}.order_items` o
JOIN `{PROJECT_ID}.{DATASET_ID}.products` p ON o.product_id = p.product_id
GROUP BY name
ORDER BY total_revenue DESC
LIMIT 15
"""

df = run_query(query=qry, project_id=PROJECT_ID)

plt.figure(figsize=(10, 6))
sns.barplot(data=df, x="total_revenue", y="name", palette="viridis")
plt.title("Top 15 Categories by Total Revenue")
plt.xlabel("Total Revenue")
plt.ylabel("Category")
plt.show()

### Review score distribution

In [ ]:
qry = f"""
SELECT c.state AS state,
AVG(TIMESTAMP_DIFF(o.delivery_customer_ts, o.purchase_ts, DAY)) AS avg_delivery_time
FROM `{PROJECT_ID}.{DATASET_ID}.orders` o
JOIN `{PROJECT_ID}.{DATASET_ID}.customers` c ON o.customer_id = c.customer_id
WHERE o.delivery_customer_ts IS NOT NULL
GROUP BY state
ORDER BY avg_delivery_time DESC
SLECT 10
"""

df = run_query(query=qry, project_id=PROJECT_ID)

## Average delivery time by state

In [ ]:
qry = f"""
SELECT c.state AS state,
AVG(TIMESTAMP_DIFF(o.delivery_customer_ts, o.purchase_ts, DAY)) AS avg_delivery_time
FROM `{PROJECT_ID}.{DATASET_ID}.orders` o
JOIN `{PROJECT_ID}.{DATASET_ID}.customers` c ON o.customer_id = c.customer_id
WHERE o.delivery_customer_ts IS NOT NULL
GROUP BY state
ORDER BY avg_delivery_time DESC
SLECT 10
"""

df = run_query(query=qry, project_id=PROJECT_ID)

plt.figure(figsize=(12, 8))
sns.barplot(data=df, x="state", y="avg_delivery_time", palette="rocket")
plt.title("Average Delivery Time by State")
plt.show()

## Customer churn

### Most recent purchase per customer

In [ ]:
qry = f"""
SELECT c.customer_uid AS customer_unique_id,
MAX(o.purchase_ts) AS last_purchase
FROM `{PROJECT_ID}.{DATASET_ID}.orders` o
JOIN `{PROJECT_ID}.{DATASET_ID}.customers` c ON o.customer_id = c.customer_id
GROUP BY customer_unique_id
"""

df_churn = run_query(query=qry, project_id=PROJECT_ID)
df_churn.head()

In [ ]:
DAYS_TO_CHURN = 90
churn_cutoff = df_churn["last_purchase"].max() - timedelta(days=DAYS_TO_CHURN)
df_churn["churned"] = df_churn["last_purchase"] < churn_cutoff

In [ ]:
churn_rate = df_churn["churned"].mean()

plt.figure(figsize=(5, 5))
sns.barplot(x=["Active", "Churned"], y=[1 - churn_rate, churn_rate], palette="mako")
plt.title("Customer Churn Rate")
plt.show()

## Customer LifeTime Value (LTV)

### Revenue and lifetime per customer

In [ ]:
qry = f"""
SELECT c.customer_uid AS customer_unique_id,
MIN(o.purchase_ts) AS first_purchase,
MAX(o.purchase_ts) AS last_purchase,
TIMESTAP_DIFF(MAX(o.purchase_ts), MIN(o.purchase_ts), DAY) AS lifetime
FROM `{PROJECT_ID}.{DATASET_ID}.orders` o
JOIN `{PROJECT_ID}.{DATASET_ID}.customers` c ON o.customer_id = c.customer_id
JOIN `{PROJECT_ID}.{DATASET_ID}.order_items` oi ON o.order_id = oi.order_id
GROUP BY customer_unique_id
"""

df = run_query(query=qry, project_id=PROJECT_ID)

plt.figure(figsize=(12, 8))
sns.histplot(df["lifetime"], bins=50, kde=True)
plt.xlabel("Total Revenue per Customer")
plt.show()

### LTV vs lifetime

In [ ]:
plt.figure(figsize=(8, 5))
sns.scatterplot(data=df, x="lifetime", y="total_revenue")
plt.title("LTV vs Customer Lifetime")
plt.show()

## Cohort analysis

In [ ]:
qry = f"""
WITH orders_ AS (
    SELECT c.customer_uid AS customer_unique_id, 
        DATE_TRUNC(o.purchase_ts, MONTH) AS order_month
    FROM `{PROJECT_ID}.{DATASET_ID}.customers` c 
    JOIN `{PROJECT_ID}.{DATASET_ID}.orders` o ON o.customer_id = c.customer_id
),
cohorts AS (
    SELECT 
        customer_unique_id,
        MIN(order_month) AS cohort_month
    FROM orders_
    GROUP BY customer_unique_id
),
cohort_activity AS (
    SELECT
        o.customer_unique_id,
        cohort_month,
        order_month,
        DATE_DIFF(order_month, cohort_month, MONTH) as month_offset
    FROM orders_ o
    JOIN cohorts c USING(customer_unique_id)
)
SELECT
    cohort_month, 
    month_offset,
    COUNT(DISTINCT customer_unique_id) AS num_customers
FROM cohort_activity
GROUP BY cohort_month, month_offset
ORDER BY cohort_month, month_offset;    
"""

df_cohort = run_query(query=qry, project_id=PROJECT_ID)

In [ ]:
cohort_table = df_cohort.pivot_table(
    index="cohort_month", columns="month_offset", values="num_customers"
)

cohort_sizes = cohort_table[0]
retention = cohort_table.divide(cohort_sizes, axis=0)

## Conclusions